In [11]:
# ! pip install xlrd

In [12]:
import urllib.request
import urllib.parse
import json
from tqdm import tqdm_notebook
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import Point

In [13]:
df = pd.read_excel('./data/물류창고정보_위치기반_231124.xls')

In [14]:
df = df.drop(['Unnamed: 5','Unnamed: 7'], axis=1)
col_name = list(df.iloc[1,:])
col_name[0:3] = df.columns[0:3]
df.columns = col_name
df = df.iloc[2:,].reset_index().drop(['index'], axis=1)
df = df.drop(['동수'], axis=1)

In [15]:
df.head()

,상호명,성명,소재지,일반창고,냉동냉장,항만창고,보세창고,보관저장업,냉동냉장,축산물보관,냉동냉장
0,삼성전자로지텍,최수영,강원특별자치도 원주시 문막읍 반계리 1093번지 1호,4039,0,0,0,0,0,0,0
1,삼성전자로지텍㈜,최수영,세종특별자치시 부강면 갈산리 470번지 2호,8264.5,0,0,0,0,0,0,0
2,씨제이대한통운㈜ 남강원지사,강신호,강원특별자치도 원주시 흥업면 사제리 1339번지 1호,2430.08,0,0,0,0,0,0,0
3,주식회사 엘엑스판토스,최원혁,강원특별자치도 원주시 문막읍 동화리 608번지 42호,2953,0,0,0,0,0,0,0
4,씨제이대한통운 주식회사,강신호,강원특별자치도 원주시 호저면 주산리 275번지,2764,0,0,0,0,0,0,0


In [16]:
# 경북 지역 창고, 울릉도 제외
gb_df = df[df['소재지'].str.contains('경상북도|대구광역')]
gb_df = gb_df[~gb_df['소재지'].str.contains('울릉')].reset_index().drop(['index'], axis=1)
gb_df

,상호명,성명,소재지,일반창고,냉동냉장,항만창고,보세창고,보관저장업,냉동냉장,축산물보관,냉동냉장
0,경동물류(주),백영길,경상북도 칠곡군 연화리 700번지 영남종합물류 화물취급장 사동,3644,0,0,0,0,0,0,0
1,합동물류(주),백영창,경상북도 칠곡군 지천면 연화리 700번지 영남복합물류 사동,1797,0,0,0,0,0,0,0
2,씨제이대한통운(주),강신호,경상북도 칠곡군 가산면 다부리 445번지,3495,0,0,0,0,0,0,0
3,롯데하이마트(주),남창희,경상북도 칠곡군 왜관읍 금산리 390번지 외 1필지,11364,0,0,0,0,0,0,0
4,씨제이대한통운(주),강신호,대구광역시 달서구 장동 306번지 111호,2049.44,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
317,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0,0,0,0,0,234,0,0
318,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0,0,0,0,0,1765,0,0
319,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0,0,0,0,0,982.06,0,0
320,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0,0,0,0,0,6654.34,0,0


In [17]:
# 중복되는 칼럼명 변경
gb_df.columns = ['상호명', '성명', '소재지', '일반창고', '냉동냉장1', '항만창고', '보세창고', '보관저장업', '냉동냉장2',
       '축산물보관', '냉동냉장3']

In [18]:
# 냉동, 냉장 창고만 필터링
gb_df.iloc[:,3:] = gb_df.iloc[:,3:].astype(float)
gb_df = gb_df[(gb_df["냉동냉장1"] != 0) | (gb_df["냉동냉장2"] != 0) | (gb_df["냉동냉장3"] != 0) | (gb_df["축산물보관"] != 0)]
gb_df

,상호명,성명,소재지,일반창고,냉동냉장1,항만창고,보세창고,보관저장업,냉동냉장2,축산물보관,냉동냉장3
9,대산농협,지용식,경상북도 김천시 대덕면 관기리 138-15,660.0,2626.24,0.0,0.0,0.0,0.0,0.0,0.0
10,김천유통영농조합법인,이기양,경상북도 김천시 대항면 이로리길 69,0.0,3427.13,0.0,0.0,0.0,0.0,0.0,0.0
11,김천유통영농조합법인,이기양,경상북도 김천시 다수동 510번지 1호,0.0,2159.59,0.0,0.0,0.0,0.0,0.0,0.0
19,한국농수산식품유통공사,김춘진,대구광역시 동구 괴전동 258번지,0.0,6634.0,0.0,0.0,0.0,0.0,0.0,0.0
20,씨제이대한통운(주),강신호,경상북도 경산시 압량면 부적리 151번지,5208.08,3178.02,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
317,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0.0,0.0,0.0,0.0,0.0,234.0,0.0,0.0
318,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0.0,0.0,0.0,0.0,0.0,1765.0,0.0,0.0
319,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0.0,0.0,0.0,0.0,0.0,982.06,0.0,0.0
320,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0.0,0.0,0.0,0.0,0.0,6654.34,0.0,0.0


In [19]:
vworld_key='82165070-4BB7-3196-88C8-EABFD5069A9C'

In [20]:
# 도로명 주소
for i in tqdm_notebook(gb_df.index):
    req_url='http://api.vworld.kr/req/address?service=address&type=road&request=getCoord&key='+vworld_key+'&address='+urllib.parse.quote(gb_df.at[i,'소재지'])
    req=urllib.request.Request(req_url)
    try:
        response=urllib.request.urlopen(req)
        if response.getcode()==200:
            json_data=json.loads(response.read())

            gb_df.at[i,'경도']=json_data['response']['result']['point']['x']
            gb_df.at[i,'위도']=json_data['response']['result']['point']['y']
        else: # TIME-OUT 400, SERVER FAIL 500
            print('응답 없음: '+str(response.getcode()))
            break
    except Exception as e: # 에러 난 것은 원인만 프린트하고 건너뛰고 다음 데이터 처리할 때 
        print('Error:%s[%s]'%(e,gb_df.at[i,'상호명']))
    else:
        print(f"Index {i} not found in gb_df.")

/var/folders/hs/qj17fzlx2zb2j3vmw0stf78r0000gn/T/ipykernel_99427/3036348063.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(gb_df.index):


  0%|          | 0/173 [00:00<?, ?it/s]

/var/folders/hs/qj17fzlx2zb2j3vmw0stf78r0000gn/T/ipykernel_99427/3036348063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gb_df.at[i,'경도']=json_data['response']['result']['point']['x']
/var/folders/hs/qj17fzlx2zb2j3vmw0stf78r0000gn/T/ipykernel_99427/3036348063.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gb_df.at[i,'위도']=json_data['response']['result']['point']['y']


Error:'result'[대산농협]
Index 10 not found in gb_df.
Error:'result'[김천유통영농조합법인]
Error:'result'[한국농수산식품유통공사]
Error:'result'[씨제이대한통운(주)]
Error:'result'[동원로엑스(주)]
Error:'result'[(주)대한냉동]
Index 29 not found in gb_df.
Error:'result'[씨제이대한통운(주)]
Index 43 not found in gb_df.
Error:'result'[씨제이대한통운(주)]
Error:'result'[대상(주)]
Error:'result'[주식회사 지에스네트웍스(경산지점)]
Error:'result'[오뚜기물류서비스(주)]
Error:'result'[오뚜기물류서비스(주)]
Error:'result'[(주)케이비경산로지스틱스위탁관리부동산투자회사]
Error:'result'[윤우상창고]
Error:'result'[도청창고]
Error:'result'[(주)유진로지스틱스]
Error:'result'[(주)오브]
Index 114 not found in gb_df.
Index 115 not found in gb_df.
Index 116 not found in gb_df.
Index 117 not found in gb_df.
Error:'result'[구룡포수협 제빙냉동공장]
Index 124 not found in gb_df.
Index 125 not found in gb_df.
Error:'result'[삼가영]
Index 127 not found in gb_df.
Index 128 not found in gb_df.
Error:'result'[(주)황산인터내셔날]
Index 130 not found in gb_df.
Index 131 not found in gb_df.
Index 132 not found in gb_df.
Index 133 not found in gb_df.
Index 134 not found in gb

In [21]:
# 지번 주소
for i in tqdm_notebook(gb_df.index):
    req_url='http://api.vworld.kr/req/address?service=address&type=parcel&request=getCoord&key='+vworld_key+'&address='+urllib.parse.quote(gb_df.at[i,'소재지'])
    req=urllib.request.Request(req_url)
    try:
        response=urllib.request.urlopen(req)
        if response.getcode()==200:
            json_data=json.loads(response.read())

            gb_df.at[i,'경도']=json_data['response']['result']['point']['x']
            gb_df.at[i,'위도']=json_data['response']['result']['point']['y']
        else: # TIME-OUT 400, SERVER FAIL 500
            print('응답 없음: '+str(response.getcode()))
            break
    except Exception as e: # 에러 난 것은 원인만 프린트하고 건너뛰고 다음 데이터 처리할 때 
        print('Error:%s[%s]'%(e,gb_df.at[i,'상호명']))
    else:
        print(f"Index {i} not found in gb_df.")

/var/folders/hs/qj17fzlx2zb2j3vmw0stf78r0000gn/T/ipykernel_99427/2983723874.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(gb_df.index):


  0%|          | 0/173 [00:00<?, ?it/s]

Index 9 not found in gb_df.
Error:'result'[김천유통영농조합법인]
Index 11 not found in gb_df.
Index 19 not found in gb_df.
Error:'result'[씨제이대한통운(주)]
Index 21 not found in gb_df.
Index 22 not found in gb_df.
Error:'result'[어신창고]
Index 39 not found in gb_df.
Error:'result'[주식회사 비지에프로지스 대구]
Error:'result'[씨제이대한통운(주)]
Index 68 not found in gb_df.
Index 70 not found in gb_df.
Index 75 not found in gb_df.
Index 76 not found in gb_df.
Index 78 not found in gb_df.
Index 90 not found in gb_df.
Index 91 not found in gb_df.
Index 98 not found in gb_df.
Index 113 not found in gb_df.
Error:'result'[경주시수산업협동조합]
Error:'result'[포항수산업협동조합제빙공장]
Error:'result'[㈜연우]
Error:'result'[(주)미식수산]
Index 118 not found in gb_df.
Error:'result'[부강가쓰오]
Error:'result'[부강가쓰오]
Index 126 not found in gb_df.
Error:'result'[(주)바다마을]
Error:'result'[(주)목양에프원]
Error:'result'[(주)황산인터내셔날]
Error:'result'[㈜에스앤비인터내셔널]
Error:'result'[대왕수산]
Error:'result'[주신수산식품(주)]
Error:'result'[진우수산]
Error:'result'[㈜세동푸드]
Error:'result'[동명씨푸드(주)]
Error:'r

In [22]:
gb_df = gb_df.dropna(subset =['경도']).reset_index().drop(['index'], axis=1)
gb_df

,상호명,성명,소재지,일반창고,냉동냉장1,항만창고,보세창고,보관저장업,냉동냉장2,축산물보관,냉동냉장3,경도,위도
0,대산농협,지용식,경상북도 김천시 대덕면 관기리 138-15,660.0,2626.24,0.0,0.0,0.0,0.0,0.0,0.0,127.97441876903079,35.92632877493597
1,김천유통영농조합법인,이기양,경상북도 김천시 대항면 이로리길 69,0.0,3427.13,0.0,0.0,0.0,0.0,0.0,0.0,128.064241142,36.126308290
2,김천유통영농조합법인,이기양,경상북도 김천시 다수동 510번지 1호,0.0,2159.59,0.0,0.0,0.0,0.0,0.0,0.0,128.0626728744702,36.12540578223084
3,한국농수산식품유통공사,김춘진,대구광역시 동구 괴전동 258번지,0.0,6634.0,0.0,0.0,0.0,0.0,0.0,0.0,128.74034526410856,35.8703443367488
4,동원로엑스(주),박성순,경상북도 경산시 진량읍 신상리 1187번지,8800.84,1854.6,0.0,0.0,0.0,0.0,0.0,0.0,128.83454615049322,35.87716877581617
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0.0,0.0,0.0,0.0,0.0,234.0,0.0,0.0,128.444990578,35.956688292
162,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0.0,0.0,0.0,0.0,0.0,1765.0,0.0,0.0,128.442743994,35.954255266
163,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0.0,0.0,0.0,0.0,0.0,982.06,0.0,0.0,128.449855727,35.961823064
164,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0.0,0.0,0.0,0.0,0.0,6654.34,0.0,0.0,129.434047806,36.100114576


In [26]:
gb_df_coord = gb_df[["상호명", "소재지", "경도", "위도"]]
# gb_df_coord.to_csv("for_compare.csv", index = False)

In [23]:
points_geometry_warehouse = gpd.GeoDataFrame(geometry=[Point(xy) for xy in zip(gb_df['경도'], gb_df['위도'])], crs="EPSG:4326")
# points_geometry_warehouse.to_file("GBDG_refrigerated_warehouse.geojson", driver='GeoJSON')
points_geometry_warehouse

,geometry
0,POINT (127.97442 35.92633)
1,POINT (128.06424 36.12631)
2,POINT (128.06267 36.12541)
3,POINT (128.74035 35.87034)
4,POINT (128.83455 35.87717)
...,...
161,POINT (128.44499 35.95669)
162,POINT (128.44274 35.95426)
163,POINT (128.44986 35.96182)
164,POINT (129.43405 36.10011)


In [24]:
from shapely import wkt
# points_df = pd.read_csv("warehouse_coord.csv")
points_df['geometry'] = points_df['geometry'].apply(wkt.loads)
points = gpd.GeoDataFrame(points_df, crs = "EPSG:4326")

NameError: name 'points_df' is not defined

In [ ]:
points

,geometry
0,POINT (128.45064 35.95743)
1,POINT (128.45064 35.95743)
2,POINT (128.52060 36.04410)
3,POINT (128.41195 35.96454)
4,POINT (128.50732 35.83935)
...,...
305,POINT (128.44499 35.95669)
306,POINT (128.44274 35.95426)
307,POINT (128.44986 35.96182)
308,POINT (129.43405 36.10011)
